In [144]:
import numpy as np
import re

In [145]:
with open('input/12-22-input', 'r') as f:
    data = [x for x in f]

###  Part 1

In [146]:
path = data[-1].strip()
board = [x.rstrip() for x in data[:-2]]

board = [
'        ...#',
'        .#..',
'        #...',
'        ....',
'...#.......#',
'........#...',
'..#....#....',
'..........#.',
'        ...#....',
'        .....#..',
'        .#......',
'        ......#.'
]

path = '10R5L5R10L4R5L5'

In [147]:
#  Set up the board and path into a more readily usable format
max_row = max(len(s) for s in board)
board = np.array([list((s + ' '*(max_row - len(s))).replace(' ','X')) for s in board])

path = [(int(re.findall(r'\d+', s)[0]), re.findall(r'[LR]', s)[0]) for s in re.findall(r'(\d+[RL])', path)]

#  Set up some initial things to start the movement process
start = (0, min(i for i in range(0, board.shape[1]) if board[0,i] == '.'))
current = start

facing = 'right'  #  Use 'right', 'left', 'up', 'down' for the facing, so that 
                  #  the directions 'L' and 'R' will determine the new facing
                  #  relative to the current one (rotating left and right)

new_facing = { ('right', 'L') : 'up', ('right', 'R') : 'down', ('left', 'L') : 'down', ('left', 'R') : 'up',
                  ('up', 'L') : 'left', ('up', 'R') : 'right', ('down', 'L') : 'right', ('down', 'R') : 'left' }
final_facing = { 'right': 0, 'down': 1, 'left': 2, 'up': 3 }

dims = board.shape

#  Perform the movement
for steps, change in path:
    for i in range(steps):
        if facing == 'right':
            new = (current[0], (current[1] + 1) % dims[1])
            if board[new] == 'X':
                new = (new[0], min(i for i in range(dims[1]) if board[new[0], i] != 'X'))
        elif facing == 'left':
            new = (current[0], (current[1] - 1) % dims[1])
            if board[new] == 'X':
                new = (new[0], max(i for i in range(dims[1]) if board[new[0], i] != 'X'))
        elif facing == 'up':
            new = ((current[0] - 1) % dims[0], current[1])
            if board[new] == 'X':
                new = (max(i for i in range(dims[0]) if board[i, new[1]] != 'X'), new[1])
        elif facing == 'down':
            new = ((current[0] + 1) % dims[0], current[1])
            if board[new] == 'X':
                new = (min(i for i in range(dims[0]) if board[i, new[1]] != 'X'), new[1])
        if board[new] == '.':
            current = new

    facing = new_facing[(facing, change)]


#  Output the "final password"
print(f'{(current[0] + 1) * 1000 + (current[1] + 1) * 4 + final_facing[facing]}')

189140


###  Part 2
Now we're walking on the cube.  We can try to just change the procedure for when you walk off the "side" of the board above.  

But I think it might be easier to rewrite the code to use six "boards", one for each side of the cube and walk on that.  We then don't have to deal with `'X'` squares, we just check if one of the indices would take you off the board.  Then depending upon the neighboring face, we can adjust the facing and continue the walk for the remaining steps.  Some "wrapping" will involve changing the facing (relative to the 2-D boards), and others won't.  

In [152]:
path = data[-1].strip()
board = [x.rstrip() for x in data[:-2]]

board = [
'        ...#',
'        .#..',
'        #...',
'        ....',
'...#.......#',
'........#...',
'..#....#....',
'..........#.',
'        ...#....',
'        .....#..',
'        .#......',
'        ......#.'
]

path = '10R5L5R10L4R5L5'

['                                                  ............#..#..................#............#.......#..............#.............................',
 '                                                  ...#...#...#.......#............................#.#.....#..#...........................#......#.....',
 '                                                  #.....#....#....#...............#..#.........#..................#......................#.##......#..',
 '                                                  ...#........#.....#...........#.......#.......###....#.......#.............##...........#.#.........',
 '                                                  ..............#.......#............................#.....#..........#....#.........................#',
 '                                                  ...................#.#........#.......#.......#.........#......................#......#.#...#.....#.',
 '                                                  ......#.#......#..

In [154]:
len(board)

200

In [151]:
#  Set up the collection of boards, i.e. the faces of the cube as a dictionary of numpy arrays

rows = len(board)
columns = len(board[0])
faces = {}
faces[1] = np.array([list(x.strip()) for x in board[:rows//3]])
for i in [2, 3, 4]:
    faces[i] = np.array([list(x[(i-2)*columns//3:(i-1)*columns//3]) for x in board[rows//3:2*rows//3]])
for i in [5, 6]:
    faces[i] = np.array([list(x.strip()[(i-5)*columns//3:(i-4)*columns//3]) for x in board[2*rows//3:rows]])


#  Location is given as (board, (row,col))
start = (1, (0,0))
current = start

facing = 'right'

change_board_facing = { 
    (1,'up'): (2, 'down'), (1,'right'): (6, 'left'), (1,'left'): (3, 'down'), (1, 'down'): (4, 'down'),
    (2,'up'): (1,'down'), (2,'right'): (3,'right'), (2,'left'): (6,'up'), (2, 'down'): (5,'up'),
    (3,'up'): (1,'right'), (3,'right'): (4,'right'), (3,'left'): (2,'left'), (3, 'down'): (5,'right'),
    (4,'up'): (1,'up'), (4,'right'): (6,'down'), (4,'left'): (3,'left'), (4,'down'): (5, 'down'),
    (5,'up'): (4,'up'), (5,'right'): (6,'right'), (5,'left'): (3,'up'), (5, 'down'): (2,'up'),
    (6,'up'): (4,'left'), (6,'right'): (1,'left'), (6,'left'): (5,'left'), (6, 'down'): (2,'right'),
} 

path = [(int(re.findall(r'\d+', s)[0]), re.findall(r'[LR]', s)[0]) for s in re.findall(r'(\d+[RL])', path)]

/var/folders/58/_jj0r88962x3gm2nywjgwr000000gn/T/ipykernel_1228/990149976.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  faces[1] = np.array([list(x.strip()) for x in board[:rows//3]])
/var/folders/58/_jj0r88962x3gm2nywjgwr000000gn/T/ipykernel_1228/990149976.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  faces[i] = np.array([list(x.strip()[(i-2)*columns//3:(i-1)*columns//3]) for x in board[rows//3:2*rows//3]])
/var/folders/58/_jj0r88962x3gm2nywjgwr000000gn/T/ipykernel_1228/990149976.py:10: VisibleDeprecationWarning: Creating an ndarray from r

TypeError: expected string or bytes-like object

In [135]:
path = [(int(re.findall(r'\d+', s)[0]), re.findall(r'[LR]', s)[0]) for s in re.findall(r'(\d+[RL])', path)]

dims = faces[1].shape[0]  #  faces are square, so let's only keep one of those dimensions

#  Perform the movement
for steps, change in path:
    for i in range(steps):
        if facing == 'right':
            new = (current[1][0], current[1][1] + 1)
            if new[1] >= dims:  #  moving off the right hand side of the current face
                if current[0] == 1:
                    new = (6, (dims - current[1][0] - 1, dims - 1))
                    facing = 'left'
                elif current[0] in [2, 3, 5]:
                    new = (current[0] + 1, (new[0], 0))
                elif current[0] == 4:
                    new = (6, (0, dims - current[1][0] - 1))
                    facing = 'down'
                elif current[0] == 6:
                    new = (1, (dims - current[1][0] - 1, dims - 1))
                    facing = 'left'
            else:
                new = (current[0], new)
        elif facing == 'left':
            new = (current[1][0], current[1][1] - 1)
            if new[1] < 0:  #  moving off the left hand side of the current face
                if current[0] == 1:
                    new = (3, (0, current[1][0]))
                    facing = 'down'
                elif current[0] in [3, 4, 6]:
                    new = (current[0] - 1, (new[0], dims - 1))
                elif current[0] == 2:
                    new = (6, (dims - current[1][0] - 1, dims - 1))
                    facing = 'up'
                elif current[0] == 5:
                    new = (3, (dims - 1, dims - current[1][0] - 1))
                    facing = 'up'
            else:
                new = (current[0], new)
        elif facing == 'up':
            new = (current[1][0] - 1, current[1][1])
            if new[0] < 0:  #  moving off the top of the current face
                if current[0] == 1:
                    new = (2, (0, dims - current[1][1] - 1))
                    facing = 'down'
                elif current[0] == 2:
                    new = (1, (dims - 1, dims - current[1][1] - 1))
                    facing = 'up'
                elif current[0] == 3:
                    new = (1, (current[1][1], 0))
                    facing = 'left'
                elif current[0] == 4:
                    new = (1, (dims - 1, current[1][1]))
                elif current[0] == 5:
                    new = (4, (dims - 1, current[1][1]))
                elif current[0] == 6:
                    new = (4, (dims - current[1][1] - 1, dims - 1))
                    facing = 'right'
            else:
                new = (current[0], new)
        elif facing == 'down':
            new = (current[1][0] + 1, current[1][1])
            if new[0] >= dims:  #  moving off the bottom of the current face
                if current[0] == 1:
                    new = (4, (0, current[1][1]))
                elif current[0] == 2:
                    new = (5, (dims - 1, dims - current[1][1] - 1))
                    facing = 'up'
                elif current[0] == 3:
                    new = (5, (dims - current[1][1] - 1, 0))
                    facing = 'right'
                elif current[0] == 4:
                    new = (5, (0, current[1][1]))
                elif current[0] == 5:
                    new = (2, (dims - 1, dims - current[1][1] - 1))
                    facing = 'up'
                elif current[0] == 6:
                    new = (2, (dims - current[1][1] - 1, 0))
                    facing = 'right'
            else:
                new = (current[0], new)

        if faces[new[0]][new[1]] == '.':
            current = new

    
    facing = new_facing[(facing, change)]

current

right (1, (0, 1))
right (1, (0, 2))
right (1, (0, 3))
right (1, (0, 3))
right (1, (0, 3))
right (1, (0, 3))
right (1, (0, 3))
right (1, (0, 3))
right (1, (0, 3))
right (1, (0, 3))
------ (1, (0, 2)) down
down (1, (1, 2))
down (1, (2, 2))
down (1, (3, 2))
down (4, (0, 2))
down (4, (1, 2))
------ (4, (1, 2)) right
right (4, (1, 3))
right (6, (0, 2))
down (6, (1, 2))
down (6, (2, 2))
down (6, (3, 2))
------ (6, (2, 2)) left
left (6, (2, 1))
left (6, (2, 0))
left (5, (2, 3))
left (5, (2, 2))
left (5, (2, 1))
left (5, (2, 1))
left (5, (2, 1))
left (5, (2, 1))
left (5, (2, 1))
left (5, (2, 1))
------ (5, (2, 2)) down
down (5, (3, 2))
down (2, (3, 1))
up (2, (2, 1))
up (2, (1, 1))
------ (2, (1, 1)) right
right (2, (1, 2))
right (2, (1, 3))
right (3, (1, 0))
right (3, (1, 1))
right (3, (1, 2))
------ (3, (1, 2)) up
2015


In [136]:
current

(3, (1, 2))

In [122]:
dims

4

In [123]:
facing

'up'

In [137]:
path

[(10, 'R'), (5, 'L'), (5, 'R'), (10, 'L'), (4, 'R'), (5, 'L')]